In [69]:
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
import re
import time
from datetime import date, datetime
import numpy as np
import random
import decimal
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import warnings
warnings.filterwarnings('ignore')

In [70]:
#Config : Provide Category to extract and pagination
amazon_listing_url = pd.DataFrame({ 'category' : ['Air Conditioner'] , 'url' : ['https://www.amazon.in/s?rh=n%3A3474656031&fs=true'] }) 
total_pages = 1

In [71]:
# Product Price, Ratings and Technical Spec Details Scraping functions

def init_browser():
    """Initialize and return a configured Chrome browser instance"""
    options = ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--incognito")
    options.add_argument("start-maximized")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    return Chrome(options=options)

def delete_cache(driver):
    driver.execute_script("window.open('')")  # Create a separate tab than the main one
    driver.switch_to.window(driver.window_handles[-1])  # Switch window to the second tab
    #driver.get('chrome://settings/clearBrowserData')  # Open your chrome settings.
    #driver.findElement(By.xpath("//*[@id='clearBrowsingDataConfirm']")).click()
    driver.execute_cdp_cmd("Network.clearBrowserCache", {})
   #perform_actions(driver, Keys.TAB * 2 + Keys.DOWN * 4 + Keys.TAB * 5 + Keys.ENTER)  # Tab to the time select and key down to say "All Time" then go to the Confirm button and press Enter
    driver.close()  # Close that window
    driver.switch_to.window(driver.window_handles[0])  # Switch Selenium controls to the original tab to continue normal functionality.

def perform_actions(driver, keys):
    actions = ActionChains(driver)
    actions.send_keys(keys)
    time.sleep(1)
    print('Performing Actions!')
    actions.perform()

# ASIN Scraping functions
def scrape_asin_urls(amazon_listing_url, total_pages):
    """Main function to scrape ASINs from listing pages"""
    final_collated_urls = pd.DataFrame()
    pages = total_pages
    for base_url in amazon_listing_url:
        all_product_urls = process_base_url(base_url, pages)
        final_collated_urls = pd.concat([final_collated_urls, all_product_urls])
        final_collated_urls = final_collated_urls.drop_duplicates('asin')
    
    final_collated_urls['Product URL'] = 'https://www.amazon.in/dp/' + final_collated_urls['asin'].astype(str)
    return final_collated_urls

def process_base_url(base_url, total_pages):
    """Process all pages for a single base URL"""
    all_product_urls = pd.DataFrame()
    
    for page in range(1, total_pages + 1):
        url = base_url + "&page=" + str(page)          
        print(url)
        browser = init_browser()
        browser.get(url)
        html = browser.page_source
        delete_cache(browser)
        browser.quit()
        time.sleep(random.uniform(0.2, 0.8))
        
        soup = BeautifulSoup(html, 'html.parser')
        page_asin_df = extract_asin_from_page(soup)
        page_asin_df['listing_url'] = base_url
        all_product_urls = pd.concat([all_product_urls, page_asin_df])
    
    return all_product_urls.reset_index(drop=True)

def extract_asin_from_page(soup):
    """Extract ASINs from a single page"""
    product_urls = []
    for div in soup.find_all("div"):
        if (data_asin := div.get("data-asin")) and (data_index := div.get("data-index")):
            product_urls.append({'asin': data_asin, 'product default order number': data_index})
    return pd.DataFrame(product_urls)

# Product Detail Scraping functions
def scrape_product_details(final_collated_urls):
    """Main function to scrape product details from product pages"""
    final_scrapped_df = pd.DataFrame()
    
    for product_url in final_collated_urls['Product URL'].tolist():
        product_df = scrape_single_product(product_url)
        product_df['Retailer'] = 'Amazon'
        final_scrapped_df = pd.concat([final_scrapped_df, product_df])
    
    return final_scrapped_df

def scrape_single_product(product_url):
    """Scrape details from a single product page"""
    product_df = pd.DataFrame({'Product URL': [product_url]})
    try:
        browser = init_browser()
        browser.get(product_url)
        html = browser.page_source
        delete_cache(browser)
        browser.quit()
        time.sleep(random.uniform(0.5, 1.05))
        
        soup = BeautifulSoup(html, 'html.parser')
        product_df = extract_basic_info(soup, product_df)
        product_df = extract_pricing_info(soup, product_df)
        product_df = extract_ratings_info(soup, product_df)
        product_df = extract_additional_info(soup, product_df)
        product_df = extract_technical_details(soup, product_df)
        
    except Exception as e:
        print(f"Error scraping {product_url}: {str(e)}")
    
    return product_df

def extract_basic_info(soup, df):
    """Extract basic product information"""
    try: df['Title'] = soup.find("span", {'id': 'productTitle'}).text.strip()
    except: df['Title'] = ''
    
    try: df['SKU Product'] = soup.find('input', {'id': 'ASIN'}).get('value')
    except: df['SKU Product'] = ''
    
    df['Scraping Date'] = date.today()
    df['Scraping Time'] = datetime.now()
    return df

def extract_pricing_info(soup, df):
    """Extract pricing-related information"""
    try:
        price = soup.find('span', {'class': 'a-price-whole'}).text.strip()
        df['Selling Price'] = float(re.sub(r'[^\d.]', '', price))
    except: df['Selling Price'] = ''
    
    try:
        mrp = soup.find("span", class_="a-price a-text-price").find("span", class_="a-offscreen").text
        df['MRP'] = float(re.sub(r'[^\d.]', '', mrp))
    except: df['MRP'] = ''
    
    try: df['Discount'] = soup.find("span", class_="a-size-large a-color-price").text.strip().replace("-", "")
    except: df['Discount'] = ''
    
    return df

def extract_ratings_info(soup, df):
    """Extract rating-related information"""
    try: df['no_ratings'] = soup.find("span", {'id': 'acrCustomerReviewText'}).text.strip()
    except: df['no_ratings'] = ''
    
    try: df['avg_rating'] = float(soup.find("span", {'class': 'reviewCountTextLinkedHistogram'}).text.split()[0])
    except: df['avg_rating'] = ''
    
    return df

def extract_additional_info(soup, df):
    """Extract additional product information"""
    try: df['Stock Status'] = soup.find("div", {'id': 'availabilityInsideBuyBox_feature_div'}).text.strip()
    except: df['Stock Status'] = ''
    
    try: df['Seller'] = soup.find("a", {'id': 'sellerProfileTriggerId'}).text.strip()
    except: df['Seller'] = ''
    
    return df

def extract_technical_details(soup, df):
    """Extract technical details from product tables"""
    try:
        table = soup.find("div", {"id": "productOverview_feature_div"}).find(
            "table", class_="a-normal a-spacing-micro"
        )
        data = []
        for tr in table.find_all("tr"):
            row = [td.text for td in tr.find_all("td")]
            data.append(row)

        table_df = pd.DataFrame(data)
        table_df = table_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
        table_df.columns = ["Attribute_Name", "Attribute_Value"]
        table_df = table_df.reset_index(drop=True)
        df = pd.concat([df, table_df], axis=1)
        col_subset = list(set(df.columns.tolist()) - set(["Attribute_Name", "Attribute_Value"]))
        df[col_subset] = df[col_subset].fillna(method="ffill")
    except Exception as e:
        print(f"Error extracting from productOverview_feature_div: {e}")

    try:
        table = soup.find("table", {"id": "productDetails_detailBullets_sections1"})
        final_product_spec_df = pd.DataFrame()
        for tr in table.find_all("tr"):
            product_value = tr.find("td").text.strip()
            product_name = tr.find("th").text.strip()
            product_spec_df = pd.DataFrame(
                {"Attribute_Name": [product_name], "Attribute_Value": [product_value]}
            )
            final_product_spec_df = pd.concat([final_product_spec_df, product_spec_df], ignore_index=True)
        df = pd.concat([df, final_product_spec_df], ignore_index=True)
        cols = df.drop(columns=["Attribute_Name", "Attribute_Value"]).columns.tolist()
        df[cols] = df[cols].fillna(method="ffill")
    except Exception as e:
        print(f"Error extracting from productDetails_detailBullets_sections1: {e}")

    try:
        table = soup.find("table", {"id": "productDetails_techSpec_section_1"})
        final_product_spec_df = pd.DataFrame()
        for tr in table.find_all("tr"):
            product_value = tr.find("td").text.strip()
            product_name = tr.find("th").text.strip()
            product_spec_df = pd.DataFrame(
                {"Attribute_Name": [product_name], "Attribute_Value": [product_value]}
            )
            final_product_spec_df = pd.concat([final_product_spec_df, product_spec_df], ignore_index=True)
        df = pd.concat([df, final_product_spec_df], ignore_index=True)
        cols = df.drop(columns=["Attribute_Name", "Attribute_Value"]).columns.tolist()
        df[cols] = df[cols].fillna(method="ffill")
    except Exception as e:
        print(f"Error extracting from productDetails_techSpec_section_1: {e}")

    return df

In [73]:
# ASIN Scraping
amazon_listing_url = amazon_listing_url['url'].unique().tolist()
final_collated_urls = scrape_asin_urls(amazon_listing_url, total_pages)
#final_collated_urls.to_csv('product_urls_amazon.csv', index=False)
final_collated_urls.head()

https://www.amazon.in/s?rh=n%3A3474656031&fs=true&page=1


,asin,product default order number,listing_url,Product URL
0,B0BK1KS6ZD,2,https://www.amazon.in/s?rh=n%3A3474656031&fs=true,https://www.amazon.in/dp/B0BK1KS6ZD
1,B0DS2DX5ZP,3,https://www.amazon.in/s?rh=n%3A3474656031&fs=true,https://www.amazon.in/dp/B0DS2DX5ZP
2,B0DQQ4XDBB,4,https://www.amazon.in/s?rh=n%3A3474656031&fs=true,https://www.amazon.in/dp/B0DQQ4XDBB
3,B0CWVDXYX1,6,https://www.amazon.in/s?rh=n%3A3474656031&fs=true,https://www.amazon.in/dp/B0CWVDXYX1
4,B0DRG7M72Z,7,https://www.amazon.in/s?rh=n%3A3474656031&fs=true,https://www.amazon.in/dp/B0DRG7M72Z


In [74]:
# Product Detail Scraping
final_scrapped_df = scrape_product_details(final_collated_urls)
#final_scrapped_df.to_csv('amazon_scrapped_data.csv', index=False)
final_scrapped_df.head()

,Product URL,Title,SKU Product,Scraping Date,Scraping Time,Selling Price,MRP,Discount,no_ratings,avg_rating,Stock Status,Seller,Attribute_Name,Attribute_Value,Retailer
0,https://www.amazon.in/dp/B0BK1KS6ZD,Daikin 1.5 Ton 3 Star Inverter Split AC (Coppe...,B0BK1KS6ZD,2025-03-01,2025-03-01 14:41:12.825522,36990.0,58400.0,,"3,950 ratings",3.9,In stock,DAWNTECH ELECTRONICS PRIVATE LIMITED,Brand,Daikin,Amazon
1,https://www.amazon.in/dp/B0BK1KS6ZD,Daikin 1.5 Ton 3 Star Inverter Split AC (Coppe...,B0BK1KS6ZD,2025-03-01,2025-03-01 14:41:12.825522,36990.0,58400.0,,"3,950 ratings",3.9,In stock,DAWNTECH ELECTRONICS PRIVATE LIMITED,Capacity,1.5 Tons,Amazon
2,https://www.amazon.in/dp/B0BK1KS6ZD,Daikin 1.5 Ton 3 Star Inverter Split AC (Coppe...,B0BK1KS6ZD,2025-03-01,2025-03-01 14:41:12.825522,36990.0,58400.0,,"3,950 ratings",3.9,In stock,DAWNTECH ELECTRONICS PRIVATE LIMITED,Cooling Power,17100 British Thermal Units,Amazon
3,https://www.amazon.in/dp/B0BK1KS6ZD,Daikin 1.5 Ton 3 Star Inverter Split AC (Coppe...,B0BK1KS6ZD,2025-03-01,2025-03-01 14:41:12.825522,36990.0,58400.0,,"3,950 ratings",3.9,In stock,DAWNTECH ELECTRONICS PRIVATE LIMITED,Special Feature,"High Ambient Operation upto 52°C, 3D Airflow, ...",Amazon
4,https://www.amazon.in/dp/B0BK1KS6ZD,Daikin 1.5 Ton 3 Star Inverter Split AC (Coppe...,B0BK1KS6ZD,2025-03-01,2025-03-01 14:41:12.825522,36990.0,58400.0,,"3,950 ratings",3.9,In stock,DAWNTECH ELECTRONICS PRIVATE LIMITED,Product Dimensions,22.9D x 88.5W x 29.8H Centimeters,Amazon
